# 연관규칙분석 (association rule)
Market Basket Analysis 이라고도 하며, Apriori 알고리즘이 가장 많이 사용된다

In [1]:
#실습
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
 
    
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)



In [3]:
df

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [2]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

In [4]:
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,0.6,(Milk)
2,0.6,(Onion)
3,0.6,(Yogurt)
4,0.6,"(Eggs, Onion)"


5개의 장바구니 중 eggs는 4번 들어가있으므로 0.8의 확률, 즉 itemsets는 물건의 조합, support는 그 물건의 조합이 들어있을 확률

In [5]:
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf


support가 0.5가 넘는 항목에 대해 향상도가 양의 상관관계에 있는것. eggs와 onion

------------

## Data load

In [1]:
import pandas as pd
import pandas as pd
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#csv파일 불러와서 dataframe화
with open("C:/Users/user/Kakao Arena/data/genre_gn_all.json", encoding = 'utf-8') as json_file: genre = json.load(json_file)
with open("C:/Users/user/Kakao Arena/data/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
with open("C:/Users/user/Kakao Arena/data/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("C:/Users/user/Kakao Arena/data/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
    
# json to dataframe
df_genre = pd.Series(genre).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
df_song_meta = pd.DataFrame(song_meta)
df_train = pd.DataFrame(train)
df_valid = pd.DataFrame(valid)


In [3]:
#songmeta에서 장르만 추출
gnr_meta=df_song_meta[['song_gn_dtl_gnr_basket','id']]

#playlist의 songs만 추출
playlist_songs=df_train['songs']

#각 playlist에 들어있는 장르만 추출 -> dataset
c=0
dataset_gnr=[]
for p in playlist_songs:
    gnr_list=[]
    for song in p:
        g=gnr_meta.loc[song,'song_gn_dtl_gnr_basket']
        
        try: #결측값 패스
            if len(g)==1:
                gnr_list.append(g[0])
            else:            
                for i in g:
                    gnr_list.append(i)
        except: pass    
    dataset_gnr.append(gnr_list)
    
    c+=1
    print(c)
    if c==30:
        break #엄청빠름

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [4]:
#print (dataset_gnr)

#association_rules 분석을 위한 one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(dataset_gnr).transform(dataset_gnr)
result = pd.DataFrame(te_ary, columns=te.columns_)

In [5]:
result.head()

,GN0101,GN0103,GN0104,GN0105,GN0201,GN0203,GN0204,GN0205,GN0301,GN0302,...,GN2504,GN2505,GN2506,GN2601,GN2602,GN2603,GN2703,GN2704,GN2901,GN2902
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,False
2,True,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,False,False,False
3,True,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,False,True,True,True,False,True,True,...,True,True,True,False,False,False,False,False,False,False


In [11]:
#apriori 적용
frequent_itemsets = apriori(result, min_support=0.3, use_colnames=True)
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GN0101),(GN0104),0.600000,0.333333,0.333333,0.555556,1.666667,0.133333,1.500000
1,(GN0104),(GN0101),0.333333,0.600000,0.333333,1.000000,1.666667,0.133333,inf
2,(GN0101),(GN0105),0.600000,0.466667,0.466667,0.777778,1.666667,0.186667,2.400000
3,(GN0105),(GN0101),0.466667,0.600000,0.466667,1.000000,1.666667,0.186667,inf
4,(GN0101),(GN0201),0.600000,0.433333,0.366667,0.611111,1.410256,0.106667,1.457143
...,...,...,...,...,...,...,...,...,...
35579,(GN0805),"(GN0101, GN0801, GN0502, GN0606, GN0509, GN050...",0.400000,0.300000,0.300000,0.750000,2.500000,0.180000,2.800000
35580,(GN0509),"(GN0101, GN0801, GN0502, GN0606, GN0805, GN050...",0.466667,0.300000,0.300000,0.642857,2.142857,0.160000,1.960000
35581,(GN0501),"(GN0101, GN0801, GN0502, GN0606, GN0805, GN050...",0.466667,0.300000,0.300000,0.642857,2.142857,0.160000,1.960000
35582,(GN0601),"(GN0101, GN0801, GN0502, GN0606, GN0805, GN050...",0.533333,0.300000,0.300000,0.562500,1.875000,0.140000,1.600000


In [10]:
frequent_itemsets = apriori(result, min_support=0.3, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.600000,(GN0101)
1,0.333333,(GN0104)
2,0.466667,(GN0105)
3,0.433333,(GN0201)
4,0.366667,(GN0204)
...,...,...
1120,0.300000,"(GN0101, GN0801, GN0502, GN0606, GN0805, GN050..."
1121,0.300000,"(GN0801, GN0502, GN0606, GN0805, GN0509, GN050..."
1122,0.300000,"(GN0801, GN0502, GN0606, GN0805, GN0509, GN050..."
1123,0.300000,"(GN0101, GN0801, GN0502, GN0606, GN0805, GN050..."
